Code chính

In [1]:
import pandas as pd
from backtesting import Backtest, Strategy
import math
from vnstock3 import Vnstock
import talib as ta

RSI_PERIOD = 14
RSI_OVERSOLD = 30
RSI_OVERBOUGHT = 70
MACD_FAST = 12
MACD_SLOW = 26
MACD_SIGNAL = 9

/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
class DCA(Strategy):
    average_monthly_income_vnd = 500  # Average monthly income in VND
    investment_percentage = 0.10  # Percentage of income to invest
    fund = 0  # Initialize the investment fund 

    def init(self):
        close = self.data.Close
        # Calculate RSI
        self.rsi = self.I(ta.RSI, close, timeperiod=14) 
        # Calculate MACD
        macd, signal_line, _ = ta.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)  
        self.macd = self.I(pd.Series, macd)
        self.signal_line = self.I(pd.Series, signal_line)
        self.previous_macd = self.I(pd.Series(macd).shift, 1)
        self.previous_signal_line = self.I(pd.Series(signal_line).shift, 1)

    def next(self):
        # Update the fund at the start of each month
        if len(self.data) % 30 == 0:  
            self.fund += self.average_monthly_income_vnd * self.investment_percentage
            
        # Check for buy signal: RSI cross above 30 and MACD cross above Signal line
        if (self.previous_macd[-1] < self.previous_signal_line[-1] and
            self.macd[-1] >= self.signal_line[-1] and
            self.rsi[-1] > 30):  
            share_price = self.data.Close[-1]
            shares_to_buy = self.fund // share_price #Buy all the shares we can with current fund
            if shares_to_buy > 0:
                self.buy(size=shares_to_buy)
                self.fund -= share_price * shares_to_buy
                
        #if (self.rsi[-1] < RSI_OVERBOUGHT and
            #self.previous_macd[-1] > self.previous_signal_line[-1] and
            #self.macd[-1] <= self.signal_line[-1]):
            #if self.position:
                #self.position.close()


def run_backtest(stock_symbol, usd_vnd_data):
    # Fetch stock data
    stock_data = Vnstock().stock(symbol=stock_symbol).quote.history(start='2019-01-01', end='2024-01-04')
    stock_data = stock_data.rename(columns={"open": "Open", "high": "High", "low": "Low", "close": "Close", "volume": "Volume"})
    stock_data.set_index('time', inplace=True)
    stock_data.index = pd.to_datetime(stock_data.index)

    # Merge USD/VND data
    stock_data.index = stock_data.index.normalize()
    stock_data['usd/vnd'] = usd_vnd_data['Close'].reindex(stock_data.index) / 1000
    stock_data['Close'] = stock_data['Close'] / stock_data['usd/vnd']
    stock_data = stock_data.dropna()

    # Run the backtest
    bt = Backtest(
        stock_data,
        DCA,
        trade_on_close=True,
    )
    stats = bt.run()
    bt.plot(filename=f'{stock_symbol}')
    
    # Calculate investment details
    trades = stats["_trades"]
    price_paid = trades["Size"] * trades["EntryPrice"]
    total_invested = price_paid.sum()

    current_shares = trades["Size"].sum()
    current_equity = current_shares * stock_data.Close.iloc[-1]

    print(f"Results for {stock_symbol}:")
    print("Total investment:", total_invested)
    print("Current Shares:", current_shares)
    print("Current Equity:", current_equity)
    print("Return:", current_equity / total_invested)
    print("-" * 50)

# Load USD/VND data
usd_vnd_data = pd.read_csv('VND=XCommon.csv')
usd_vnd_data['Date'] = pd.to_datetime(usd_vnd_data['Date'])
usd_vnd_data.set_index('Date', inplace=True)

# List of stock symbols
stock_symbols = ['KDC', 'VNM', 'MSN'] 

# Run backtest for each stock
for symbol in stock_symbols:
    run_backtest(symbol, usd_vnd_data)

2024-08-09 20:03:17,018 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:456: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; u

Results for KDC:
Total investment: 1998.0808314313242
Current Shares: 1380
Current Equity: 3517.368961973279
Return: 1.760373707931332
--------------------------------------------------


/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:456: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:659: UserWarnin

Results for VNM:
Total investment: 1998.3488365501757
Current Shares: 610
Current Equity: 1708.250770811922
Return: 0.8548311183551612
--------------------------------------------------


/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:456: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:659: UserWarnin

Results for MSN:
Total investment: 1998.6061032305056
Current Shares: 649
Current Equity: 1838.2774922918811
Return: 0.919779785181546
--------------------------------------------------


DƯỚI NÀY LÀ TEST THÔI

In [3]:
class DCA(Strategy):
    average_monthly_income_vnd = 500  # Average monthly income in VND
    investment_percentage = 0.10  # Percentage of income to invest
    fund = 0  # Initialize the investment fund

    def init(self):
        close = self.data.Close
        # Calculate RSI
        self.rsi = self.I(ta.RSI, close, timeperiod=14) 
        # Calculate MACD
        macd, signal_line, _ = ta.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)  
        self.macd = self.I(pd.Series, macd)
        self.signal_line = self.I(pd.Series, signal_line)
        self.previous_macd = self.I(pd.Series(macd).shift, 1)
        self.previous_signal_line = self.I(pd.Series(signal_line).shift, 1)

    def next(self):
        # Update the fund at the start of each month
        if len(self.data) % 30 == 0:  
            self.fund += self.average_monthly_income_vnd * self.investment_percentage
            print(f"Day: {self.data.index[-1]} total fund: {self.fund}")

        # Check for buy signal
        if (self.previous_macd[-1] < self.previous_signal_line[-1] and
            self.macd[-1] >= self.signal_line[-1] and
            self.rsi[-1] > 30):  
            
            share_price = self.data.Close[-1]
            shares_to_buy = self.fund // share_price
            if shares_to_buy > 0:
                self.buy(size=shares_to_buy)
                self.fund -= share_price * shares_to_buy
                print(f"Buy executed at {self.data.index[-1]} with {shares_to_buy} shares at price {share_price}, total price {share_price * shares_to_buy}")

        #if (self.rsi[-1] < RSI_OVERBOUGHT and
            #self.previous_macd[-1] > self.previous_signal_line[-1] and
            #self.macd[-1] <= self.signal_line[-1]):
            #if self.position:
                #self.position.close()


def run_backtest(stock_symbol, usd_vnd_data):
    # Fetch stock data
    stock_data = Vnstock().stock(symbol=stock_symbol).quote.history(start='2019-01-01', end='2024-01-04')
    stock_data = stock_data.rename(columns={"open": "Open", "high": "High", "low": "Low", "close": "Close", "volume": "Volume"})
    stock_data.set_index('time', inplace=True)
    stock_data.index = pd.to_datetime(stock_data.index)

    # Merge USD/VND data
    stock_data.index = stock_data.index.normalize()
    stock_data['usd/vnd'] = usd_vnd_data['Close'].reindex(stock_data.index) / 1000
    stock_data['Close'] = stock_data['Close'] / stock_data['usd/vnd']
    stock_data = stock_data.dropna()

    # Run the backtest
    bt = Backtest(
        stock_data,
        DCA,
        trade_on_close=True,
    )
    stats = bt.run()
    bt.plot(filename=f'{stock_symbol}')
    print(stats)

# Load USD/VND data
usd_vnd_data = pd.read_csv('VND=XCommon.csv')
usd_vnd_data['Date'] = pd.to_datetime(usd_vnd_data['Date'])
usd_vnd_data.set_index('Date', inplace=True)

# List of stock symbols
stock_symbols = ['KDC']  # Assume more stocks listed if needed

# Run backtest for each stock
for symbol in stock_symbols:
    run_backtest(symbol, usd_vnd_data)

2024-08-09 20:03:28,031 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


Day: 2019-04-02 00:00:00 total fund: 50.0
Buy executed at 2019-04-05 00:00:00 with 62.0 shares at price 0.8052418311923442, total price 49.92499353392534
Day: 2019-05-20 00:00:00 total fund: 50.07500646607466
Buy executed at 2019-05-31 00:00:00 with 70.0 shares at price 0.7120649264679745, total price 49.84454485275821
Day: 2019-07-02 00:00:00 total fund: 50.23046161331645
Day: 2019-08-13 00:00:00 total fund: 100.23046161331645
Buy executed at 2019-08-15 00:00:00 with 150.0 shares at price 0.6666092127375361, total price 99.99138191063042
Day: 2019-09-25 00:00:00 total fund: 50.23907970268603
Buy executed at 2019-10-14 00:00:00 with 64.0 shares at price 0.7735488008038965, total price 49.50712325144938
Day: 2019-11-06 00:00:00 total fund: 50.73195645123665
Day: 2019-12-18 00:00:00 total fund: 100.73195645123664
Buy executed at 2019-12-25 00:00:00 with 138.0 shares at price 0.7293599758318587, total price 100.65167666479651
Day: 2020-02-06 00:00:00 total fund: 50.08027978644013
Buy exec

/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:456: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/Users/mac/anaconda3/lib/python3.11/site-packages/backtesting/_plotting.py:659: UserWarnin

Start                     2019-01-02 00:00:00
End                       2024-01-03 00:00:00
Duration                   1827 days 00:00:00
Exposure Time [%]                   94.964029
Equity Final [$]                 11528.712244
Equity Peak [$]                  11969.525414
Return [%]                          15.287122
Buy & Hold Return [%]              192.659835
Return (Ann.) [%]                    2.907032
Volatility (Ann.) [%]                4.010097
Sharpe Ratio                         0.724928
Sortino Ratio                        1.223472
Calmar Ratio                         0.401546
Max. Drawdown [%]                   -7.239608
Avg. Drawdown [%]                   -0.843238
Max. Drawdown Duration      523 days 00:00:00
Avg. Drawdown Duration       48 days 00:00:00
# Trades                                   28
Win Rate [%]                             75.0
Best Trade [%]                     335.418154
Worst Trade [%]                     -4.459119
Avg. Trade [%]                    

In [4]:
kdc_data = Vnstock().stock(symbol="KDC").quote.history(start='2019-01-01', end='2024-01-04')
kdc_data = kdc_data.rename(columns={"open": "Open", "high": "High", "low": "Low", "close": "Close", "volume": "Volume"})
kdc_data.set_index('time', inplace=True)
kdc_data.index = pd.to_datetime(kdc_data.index)
usd_vnd_data = pd.read_csv('VND=XCommon.csv')
usd_vnd_data['Date'] = pd.to_datetime(usd_vnd_data['Date'])
usd_vnd_data.set_index('Date', inplace=True)
# Merge USD/VND data with stock data
kdc_data.index = kdc_data.index.normalize()
kdc_data['usd/vnd'] = usd_vnd_data['Close'].reindex(kdc_data.index) / 1000
kdc_data['Close'] = kdc_data['Close'] / kdc_data['usd/vnd']
kdc_data = kdc_data.dropna()
kdc_data

2024-08-09 20:03:33,674 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


,Open,High,Low,Close,Volume,usd/vnd
time,,,,,,
2019-01-02,20.46,20.46,19.44,0.870915,40160,23.194
2019-01-03,20.20,20.20,18.81,0.838148,95690,23.194
2019-01-04,19.44,19.44,18.64,0.834699,61560,23.194
2019-01-07,19.36,19.70,18.77,0.834699,104110,23.194
2019-01-08,18.89,19.32,18.81,0.830818,57270,23.194
...,...,...,...,...,...,...
2023-12-27,62.39,62.69,61.70,2.548818,808708,24.325
2023-12-28,62.39,62.98,61.80,2.546724,761168,24.345
2023-12-29,61.90,62.00,61.80,2.558283,752418,24.235


In [5]:
kdc_data['RSI'] = ta.RSI(kdc_data['Close'], timeperiod=RSI_PERIOD)
# Calculate MACD
kdc_data['MACD'], kdc_data['Signal_Line'], _ = ta.MACD(
    kdc_data['Close'],
    fastperiod=MACD_FAST,
    slowperiod=MACD_SLOW,
    signalperiod=MACD_SIGNAL
)
kdc_data['Previous_MACD'] = kdc_data['MACD'].shift(1)
kdc_data['Previous_Signal_Line'] = kdc_data['Signal_Line'].shift(1)
kdc_data

,Open,High,Low,Close,Volume,usd/vnd,RSI,MACD,Signal_Line,Previous_MACD,Previous_Signal_Line
time,,,,,,,,,,,
2019-01-02,20.46,20.46,19.44,0.870915,40160,23.194,NaN,NaN,NaN,NaN,NaN
2019-01-03,20.20,20.20,18.81,0.838148,95690,23.194,NaN,NaN,NaN,NaN,NaN
2019-01-04,19.44,19.44,18.64,0.834699,61560,23.194,NaN,NaN,NaN,NaN,NaN
2019-01-07,19.36,19.70,18.77,0.834699,104110,23.194,NaN,NaN,NaN,NaN,NaN
2019-01-08,18.89,19.32,18.81,0.830818,57270,23.194,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2023-12-27,62.39,62.69,61.70,2.548818,808708,24.325,50.202923,0.001570,-0.001653,0.001750,-0.002458
2023-12-28,62.39,62.98,61.80,2.546724,761168,24.345,49.734553,0.001245,-0.001073,0.001570,-0.001653
2023-12-29,61.90,62.00,61.80,2.558283,752418,24.235,52.376005,0.001897,-0.000479,0.001245,-0.001073


In [6]:
# Add a column to identify signals based on combined RSI and MACD strategy
def macd_rsi_strategy(df):
    if df.empty:
        return df
    
    df['Signal'] = 0

    # Buy signals: RSI cross above 30 and MACD cross above Signal line
    df.loc[
        (df['Previous_MACD'] < df['Previous_Signal_Line']) &
        (df['MACD'] >= df['Signal_Line']) &
        (df['RSI'] > RSI_OVERSOLD), 'Signal'] = 1

    # Sell Signals: 
    df.loc[
        (df['RSI'] < RSI_OVERBOUGHT) &
        (df['Previous_MACD'] > df['Previous_Signal_Line']) &
        (df['MACD'] <= df['Signal_Line']), 'Signal'] = -1

    return df

kdc_data = macd_rsi_strategy(kdc_data)
kdc_data = kdc_data.drop(columns=['MACD', 'Signal_Line', 'Previous_MACD', 'Previous_Signal_Line','RSI'])
kdc_data

,Open,High,Low,Close,Volume,usd/vnd,Signal
time,,,,,,,
2019-01-02,20.46,20.46,19.44,0.870915,40160,23.194,0
2019-01-03,20.20,20.20,18.81,0.838148,95690,23.194,0
2019-01-04,19.44,19.44,18.64,0.834699,61560,23.194,0
2019-01-07,19.36,19.70,18.77,0.834699,104110,23.194,0
2019-01-08,18.89,19.32,18.81,0.830818,57270,23.194,0
...,...,...,...,...,...,...,...
2023-12-27,62.39,62.69,61.70,2.548818,808708,24.325,0
2023-12-28,62.39,62.98,61.80,2.546724,761168,24.345,0
2023-12-29,61.90,62.00,61.80,2.558283,752418,24.235,0


In [7]:
kdc_data.to_csv("kdc_signal.csv")